In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from tqdm import tqdm
import ipywidgets as widgets
import time
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import re
from selenium.webdriver.support.ui import Select

In [3]:
import pandas as pd
import numpy as np
import mysql.connector

In [4]:
options = webdriver.ChromeOptions()

prefs = {"download.default_directory" : "/home/lim/dev_ws/eda/data", "download.prompt_for_download":False}
options.add_experimental_option("prefs", prefs)

url = "https://www.kobis.or.kr/kobis/business/stat/boxs/findYearlyBoxOfficeList.do"
driver = webdriver.Chrome(service=Service("/home/lim/dev_ws/eda/driver/chromedriver"), options=options)

driver.get(url)
driver.set_window_size(1800, 1300)

In [5]:
year = driver.find_element(By.ID, "sSearchYearFrom")
years = []
years_list = year.find_elements(By.TAG_NAME, "option")
for yearly in years_list:
    yearly = yearly.get_attribute("value")
    years.append(yearly)
years

['2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019',
 '2020',
 '2021',
 '2022',
 '2023',
 '2024',
 '2025']

In [6]:
# year.send_keys(years[-1])

In [10]:
movie_names = []
page = driver.page_source
soup = BeautifulSoup(page, "html.parser")
movies = soup.find("table", "tbl_comm th_sort")
moviesList = movies.find_all("span", class_="ellip per90")
for movie in moviesList:
    movie = movie.find("a").text
    movie_names.append(movie)

movie_names, len(movie_names)

(['하얼빈',
  '보고타: 마지막 기회의 땅',
  '소방관',
  '수퍼 소닉3',
  '뽀로로 극장판 바닷속 대모험',
  '무파사: 라이온 킹',
  '모아나 2',
  '극장판 짱구는 못말려: 우리들의 공룡일기',
  '위키드',
  '시빌 워: 분열의 시대',
  '러브레터',
  '서브스턴스',
  '더 폴: 디렉터스 컷',
  '해리 포터와 죽음의 성물1',
  '니코: 오로라 원정대의 모험',
  '이처럼 사소한 것들',
  '색, 계',
  '이터널 선샤인',
  '밀레니엄 맘보',
  '퍼스트레이디',
  '대가족',
  '극장판 주술회전 0',
  '나야, 문희',
  '총을 든 스님',
  '티파니에서 아침을',
  '정동원 성탄총동원 더 무비',
  '히든페이스',
  '삶과 죽음의 문제',
  '클로즈 유어 아이즈',
  '아키라',
  '룸 넥스트 도어',
  '아노라',
  '리얼리티+',
  '<에이에스엠알> 고요하게, 새벽의 산사',
  '알레고리, 잇츠 낫 미',
  '모든 것은 아르망에서 시작되었다',
  '힘을 낼 시간',
  '태양은 가득히',
  '더 퍼스트 슬램덩크',
  '아라비아의 로렌스',
  '크리스마스에 기적을 만날 확률',
  '전장의 크리스마스',
  '그랜드 부다페스트 호텔',
  '류이치 사카모토: 오퍼스',
  '쇼잉 업',
  '대함정',
  '캐롤',
  '에브리씽 에브리웨어 올 앳 원스',
  '움베르토 에코. 세계의 도서관',
  '룩백'],
 50)

In [8]:
movie_viewers = []
movieL = driver.find_elements(By.ID, "td_audiAcc")
for movie in movieL:
    movie_viewers.append(movie.text)

movie_viewers

['340,501',
 '93,154',
 '80,830',
 '75,916',
 '58,882',
 '41,753',
 '35,259',
 '28,242',
 '20,529',
 '17,822',
 '14,957',
 '4,034',
 '3,751',
 '3,074',
 '1,508',
 '1,290',
 '1,156',
 '1,073',
 '710',
 '490',
 '488',
 '482',
 '357',
 '322',
 '291',
 '290',
 '217',
 '208',
 '140',
 '115',
 '112',
 '105',
 '95',
 '89',
 '68',
 '61',
 '57',
 '53',
 '52',
 '51',
 '51',
 '48',
 '44',
 '39',
 '39',
 '35',
 '30',
 '29',
 '27',
 '25']

In [9]:
movie_genre = []
for movie in tqdm(movie_names):
    link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.LINK_TEXT, movie)))
    link.click()

    page = driver.page_source
    soup = BeautifulSoup(page, "html.parser")

    for dt in soup.find_all("dt", text="요약정보"):
        partDets = []
        dd = dt.find_next("dd")  # Get the associated <dd> element
        if dd:
            # Extract and clean the genre information
            raw_text = dd.get_text(separator="|").strip()  # Use '|' as separator for clarity
            parts = raw_text.split("|")  # Split by '|'
            genres = [
                '미스터리', '공포(호러)', '액션', '범죄', '판타지', '드라마', 'SF', '다큐멘터리',
                '코미디', '뮤지컬', '애니메이션', '스릴러', '어드벤처', '사극', '멜로/로맨스', '기타'
            ]
            for part in parts:
                part = part.strip()
                items = re.split((","), part)
                for item in items:
                    item = item.strip()
                    if item in genres :
                        partDets.append(item)
            movie_genre.append(partDets)

    close = driver.find_element(By.CLASS_NAME, "close")
    close.click()

movie_genre, len(movie_genre)

  0%|          | 0/50 [00:00<?, ?it/s]/tmp/ipykernel_45178/4026965249.py:9: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  for dt in soup.find_all("dt", text="요약정보"):
100%|██████████| 50/50 [01:10<00:00,  1.42s/it]


([['드라마'],
  ['범죄', '드라마'],
  ['드라마'],
  ['애니메이션'],
  ['애니메이션'],
  ['애니메이션'],
  ['애니메이션'],
  ['애니메이션'],
  ['판타지', '뮤지컬'],
  ['액션', '드라마'],
  ['멜로/로맨스', '드라마'],
  ['스릴러'],
  ['드라마', '판타지'],
  ['미스터리', '판타지'],
  ['애니메이션'],
  ['드라마'],
  ['멜로/로맨스'],
  ['멜로/로맨스'],
  ['드라마'],
  ['다큐멘터리'],
  ['드라마', '코미디'],
  ['애니메이션'],
  ['드라마', '코미디', '판타지'],
  ['드라마', '코미디', '기타'],
  ['드라마', '멜로/로맨스'],
  [],
  ['스릴러'],
  ['멜로/로맨스', '판타지', '코미디'],
  ['드라마'],
  ['애니메이션', 'SF'],
  ['드라마'],
  ['드라마', '코미디', '멜로/로맨스'],
  ['SF', '멜로/로맨스', '드라마'],
  ['기타'],
  ['기타'],
  [],
  ['드라마'],
  ['드라마'],
  ['드라마'],
  ['범죄', '스릴러'],
  ['애니메이션'],
  ['드라마'],
  ['애니메이션', '드라마', '액션'],
  ['드라마'],
  ['미스터리', '어드벤처'],
  ['다큐멘터리'],
  ['드라마'],
  ['액션', '어드벤처'],
  ['드라마'],
  ['액션', '코미디'],
  ['다큐멘터리'],
  ['애니메이션']],
 52)

In [11]:
movie_names = movie_names[:20]
movie_genre = movie_genre[:20]

In [12]:
len(movie_names), len(movie_genre)

(20, 20)

In [20]:
data = {"Name": movie_names, "Genre":movie_genre}
df = pd.DataFrame(data)
df.head()

,Name,Genre
0,하얼빈,[드라마]
1,보고타: 마지막 기회의 땅,"[범죄, 드라마]"
2,소방관,[드라마]
3,수퍼 소닉3,[애니메이션]
4,뽀로로 극장판 바닷속 대모험,[애니메이션]


In [21]:
for idx, data in df.iterrows():
    data["Genre"] = ', '.join(data["Genre"])
    df["Genre"][idx] = data["Genre"]

/tmp/ipykernel_45178/3355576385.py:3: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df["Genre"][idx] = data["Genre"]


In [22]:
df.head()

,Name,Genre
0,하얼빈,드라마
1,보고타: 마지막 기회의 땅,"범죄, 드라마"
2,소방관,드라마
3,수퍼 소닉3,애니메이션
4,뽀로로 극장판 바닷속 대모험,애니메이션


In [24]:
# aws key && pwd -> connection

In [27]:
cur.close()

True

In [28]:
remote.close()

In [25]:
cur.execute("CREATE TABLE movie2015 (name varchar(16), genre varchar(32))")

In [26]:
cur = remote.cursor(buffered=True)
sql = "INSERT INTO movie2015 VALUES (%s, %s)"
for i, row in df.iterrows():
    cur.execute(sql, tuple(row))
    print(tuple(row))
    remote.commit()

('하얼빈', '드라마')
('보고타: 마지막 기회의 땅', '범죄, 드라마')
('소방관', '드라마')
('수퍼 소닉3', '애니메이션')
('뽀로로 극장판 바닷속 대모험', '애니메이션')
('무파사: 라이온 킹', '애니메이션')
('모아나 2', '애니메이션')
('극장판 짱구는 못말려: 우리들의 공룡일기', '애니메이션')
('위키드', '판타지, 뮤지컬')
('시빌 워: 분열의 시대', '액션, 드라마')
('러브레터', '멜로/로맨스, 드라마')
('서브스턴스', '스릴러')
('더 폴: 디렉터스 컷', '드라마, 판타지')
('해리 포터와 죽음의 성물1', '미스터리, 판타지')
('니코: 오로라 원정대의 모험', '애니메이션')
('이처럼 사소한 것들', '드라마')
('색, 계', '멜로/로맨스')
('이터널 선샤인', '멜로/로맨스')
('밀레니엄 맘보', '드라마')
('퍼스트레이디', '다큐멘터리')


In [ ]:
movie_names = []
movie_viewers = []
movie_genre = []
movie_years = []
movieG = []
years = []


options = webdriver.ChromeOptions()

prefs = {"download.default_directory" : "/home/lim/dev_ws/eda/data", "download.prompt_for_download":False}
options.add_experimental_option("prefs", prefs)

url = "https://www.kobis.or.kr/kobis/business/stat/boxs/findYearlyBoxOfficeList.do"
driver = webdriver.Chrome(service=Service("/home/lim/dev_ws/eda/driver/chromedriver"), options=options)

driver.get(url)
driver.set_window_size(1800, 1300)

year_dropdown = driver.find_element(By.ID, "sSearchYearFrom")
select = Select(year_dropdown)  # Use Select to interact with the dropdown
years_list = select.options  # Get all options in the dropdown


for option in years_list:
    years.append(option.get_attribute("value"))

# Limit to the last few years (example: from the 10th year onward)
def dataCrawling(yearly):
    year_dropdown = driver.find_element(By.ID, "sSearchYearFrom")
    select = Select(year_dropdown)  # Re-initialize the Select object
    
    driver.execute_script("window.scrollTo(0, 0)")
    select.select_by_value(yearly)  # Select the year from the dropdown
    
    # Click the "show" button to load the data
    show = driver.find_element(By.CLASS_NAME, "wrap_btn")
    show.click()
    time.sleep(1)

    movieN = []
    # Wait for the page to load and parse it
    page = driver.page_source
    soup = BeautifulSoup(page, "html.parser")
    movies = soup.find("table", "tbl_comm th_sort")
    moviesList = movies.find_all("span", class_="ellip per90")
    for movie in moviesList:
        movie = movie.find("a").text
        movie_names.append(movie)
        movieN.append(movie)
        movie_years.append(yearly)
    
    movieL = driver.find_elements(By.ID, "td_audiAcc")
    for movie in movieL:
        movie_viewers.append(movie.text)

    for movie in tqdm(movieN):
        movieG = []
        link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.LINK_TEXT, movie)))
        link.click()

        page = driver.page_source
        soup = BeautifulSoup(page, "html.parser")

        for dt in soup.find_all("dt", text="요약정보"):
            partDets = []
            dd = dt.find_next("dd")  # Get the associated <dd> element
            if dd:
                # Extract and clean the genre information
                raw_text = dd.get_text(separator="|").strip()  # Use '|' as separator for clarity
                parts = raw_text.split("|")  # Split by '|'
                genres = [
                    '미스터리', '공포(호러)', '액션', '범죄', '판타지', '드라마', 'SF', '다큐멘터리', '전쟁',
                    '코미디', '뮤지컬', '애니메이션', '스릴러', '어드벤처', '사극', '멜로/로맨스', '기타'
                ]
                for part in parts:
                    part = part.strip()
                    items = re.split((","), part)
                    for item in items:
                        item = item.strip()
                        if item in genres :
                            partDets.append(item)
                movie_genre.append(partDets)

        close = driver.find_element(By.CLASS_NAME, "close")
        close.click()
        
    driver.close()

(0, 0, 0, 0)

In [75]:
len(movie_names), len(movie_viewers), len(movie_genre), len(movie_years)

(100, 100, 100, 100)

In [34]:
movie_genre, len(movie_genre)

([['드라마'],
  ['판타지', '드라마'],
  ['액션'],
  ['액션', '어드벤처', 'SF'],
  ['범죄', '액션'],
  ['액션', '드라마'],
  ['액션'],
  ['범죄', '드라마'],
  ['멜로/로맨스', '판타지', '뮤지컬'],
  ['액션', '어드벤처', '코미디'],
  ['액션', '어드벤처', '판타지', 'SF'],
  ['범죄'],
  ['액션', '드라마'],
  ['사극', '드라마'],
  ['액션', '어드벤처', '판타지'],
  ['액션', '범죄', '스릴러'],
  ['애니메이션', '드라마', '멜로/로맨스'],
  ['애니메이션'],
  ['코미디'],
  ['액션', '어드벤처', '판타지', '코미디'],
  ['범죄', '액션'],
  ['액션', '드라마', '스릴러', '전쟁'],
  ['액션', 'SF'],
  ['범죄', '스릴러'],
  ['액션', '어드벤처', 'SF'],
  ['범죄', '코미디'],
  ['범죄', '액션'],
  ['애니메이션', '코미디'],
  ['드라마'],
  ['드라마'],
  ['애니메이션'],
  ['액션', '범죄'],
  ['액션', 'SF'],
  ['액션', '어드벤처', '판타지', 'SF'],
  ['공포(호러)', '미스터리', '스릴러', '드라마'],
  ['액션', '어드벤처', '드라마', 'SF'],
  ['드라마'],
  ['공포(호러)'],
  ['다큐멘터리'],
  ['액션', '어드벤처', '판타지', 'SF'],
  ['액션', '코미디'],
  ['액션', '어드벤처', '판타지'],
  ['스릴러'],
  ['코미디', '어드벤처', '액션'],
  ['코미디'],
  ['미스터리', '스릴러'],
  ['공포(호러)', '미스터리', '스릴러'],
  ['범죄', '드라마'],
  ['드라마'],
  ['SF', '스릴러'],
  ['판타지', '드라마'],
  ['액션', '어드벤처', '판타지', '

In [22]:
movie_names = []
movie_viewers = []
movie_genre = []
movie_years = []
movieG = []

options = webdriver.ChromeOptions()

prefs = {"download.default_directory" : "/home/lim/dev_ws/eda/data", "download.prompt_for_download":False}
options.add_experimental_option("prefs", prefs)

url = "https://www.kobis.or.kr/kobis/business/stat/boxs/findYearlyBoxOfficeList.do"
driver = webdriver.Chrome(service=Service("/home/lim/dev_ws/eda/driver/chromedriver"), options=options)

driver.get(url)
driver.set_window_size(1800, 1300)
time.sleep(1)

year_dropdown = driver.find_element(By.ID, "sSearchYearFrom")
select = Select(year_dropdown)  # Use Select to interact with the dropdown
years_list = select.options  # Get all options in the dropdown

years = []
for option in years_list:
    years.append(option.get_attribute("value"))

# Limit to the last few years (example: from the 10th year onward)
# Loop through the years and perform scraping
for yearly in tqdm(years):
    dataCrawling(yearly)

len(movie_names), len(movie_viewers), len(movie_genre), len(movie_years)

  0%|          | 0/21 [00:00<?, ?it/s]


TypeError: dataCrawling() takes 0 positional arguments but 1 was given

In [12]:
movie_genre

[['코미디', '드라마'],
 ['SF', '액션'],
 ['사극', '드라마'],
 ['액션', '어드벤처', 'SF'],
 ['액션'],
 ['액션', '코미디', '드라마'],
 ['드라마'],
 ['스릴러'],
 ['범죄', '기타'],
 ['범죄', '액션'],
 ['드라마', '스릴러', 'SF', '액션', '어드벤처'],
 ['범죄', '액션'],
 ['코미디'],
 ['액션'],
 ['SF', '드라마'],
 ['드라마'],
 ['드라마', '어드벤처', '액션'],
 ['액션', '어드벤처', '판타지'],
 ['액션', '코미디', '범죄'],
 ['액션'],
 ['멜로/로맨스', '코미디'],
 ['범죄', '액션', '스릴러'],
 ['드라마'],
 ['드라마'],
 ['액션', 'SF', '어드벤처'],
 ['액션', '스릴러'],
 ['공포(호러)', '스릴러'],
 ['SF'],
 ['액션', '스릴러'],
 ['어드벤처', '드라마', '판타지'],
 ['스릴러'],
 ['애니메이션'],
 ['멜로/로맨스', '코미디'],
 ['액션', '어드벤처', 'SF'],
 ['액션'],
 ['스릴러', '드라마'],
 ['드라마'],
 ['드라마', '액션'],
 ['드라마'],
 ['액션', 'SF', '어드벤처'],
 ['어드벤처', '드라마'],
 ['액션', 'SF'],
 ['드라마', '멜로/로맨스'],
 ['액션', '범죄', '스릴러'],
 ['액션', '코미디'],
 ['코미디', '드라마'],
 ['멜로/로맨스', '코미디'],
 ['드라마', '액션'],
 ['SF', '액션'],
 ['공포(호러)', '스릴러'],
 ['사극', '액션'],
 ['애니메이션', '어드벤처', '코미디', '뮤지컬', '판타지'],
 ['SF'],
 ['어드벤처'],
 ['드라마'],
 ['드라마'],
 ['드라마'],
 ['액션', 'SF', '어드벤처'],
 ['사극', '액션'],
 ['액션', 'SF'],
 ['액션', '판타지

In [10]:
data = {"Name" : movie_names, "Year" : movie_years, "Genre" : movie_genre, "Viewers" : movie_viewers}
df = pd.DataFrame(data)
df.head()

,Name,Year,Genre,Viewers
0,7번방의 선물,2013,"[코미디, 드라마]","12,811,206"
1,설국열차,2013,"[SF, 액션]","9,349,993"
2,관상,2013,"[사극, 드라마]","9,134,238"
3,아이언맨 3,2013,"[액션, 어드벤처, SF]","9,001,331"
4,베를린,2013,[액션],"7,166,290"


In [11]:
df.to_csv("/home/lim/dev_ws/eda/ros_eda_project/data/movie_genre_data.csv", sep=',', encoding="utf-8")